In [0]:
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import csv
import argparse as ap
import math

In [0]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/Computer_Vision_final/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
#this is the stuff for testing 
csv_path = data_dir + "face_labels.csv"
df = pd.read_csv(csv_path)

In [0]:
df.head()
col_file = list(df.file)
col_file = col_file[:-2]

In [0]:
images = list()
labels = list()
primary_path = data_dir + "region_maps/"
#sub_dir_list = os.listdir(primary_path)
for i in range(len(col_file)):
  label = int(df['gender'][i])
  holder = col_file[i].split(".")
  name = holder[0] + ".jpg"
  path = data_dir + "region_maps/" + name
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
  image_data = np.array(img)
  images.append(image_data)
  labels.append(label)
# x = np.array(images)
test_features = np.array(images)
labels = np.array(labels)
# y = np.array(keras.utils.to_categorical(np.array(labels), num_classes=2))
test_labels = np.array(keras.utils.to_categorical(np.array(labels), num_classes=2))
# train_features , test_features ,train_labels, test_labels = train_test_split( x , y , test_size=0.4 )
print("done")

done


We have just loaded the contour maps in. Now what we want to do is to create a NN

FIRST WE WILL LOAD IN OUR TRAIN DATA>> SORRY IT IS BACKWARD!


In [0]:
#this is the stuff for testing 
csv_path_train = data_dir + "face_train.csv"
# csv_path_train = data_dir + "face_labels.csv"
df_train = pd.read_csv(csv_path_train)
col_file_train = list(df_train.file)

In [0]:
col_file_train

['face_588.jpg',
 'face_1003.jpg',
 'face_1000.jpg',
 'face_1001.jpg',
 'face_1006.jpg',
 'face_1010.jpg',
 'face_1007.jpg',
 'face_1004.jpg',
 'face_1005.jpg',
 'face_1011.jpg',
 'face_1018.jpg',
 'face_1012.jpg',
 'face_1014.jpg',
 'face_1016.jpg',
 'face_1020.jpg',
 'face_1022.jpg',
 'face_1021.jpg',
 'face_1023.jpg',
 'face_1019.jpg',
 'face_1024.jpg',
 'face_1025.jpg',
 'face_1026.jpg',
 'face_1028.jpg',
 'face_1029.jpg',
 'face_1030.jpg',
 'face_1033.jpg',
 'face_1035.jpg',
 'face_1034.jpg',
 'face_1032.jpg',
 'face_1036.jpg',
 'face_1041.jpg',
 'face_1037.jpg',
 'face_1038.jpg',
 'face_1045.jpg',
 'face_1048.jpg',
 'face_1050.jpg',
 'face_1046.jpg',
 'face_1047.jpg',
 'face_1051.jpg',
 'face_1053.jpg',
 'face_1055.jpg',
 'face_1054.jpg',
 'face_1056.jpg',
 'face_1061.jpg',
 'face_1059.jpg',
 'face_1065.jpg',
 'face_1058.jpg',
 'face_1060.jpg',
 'face_1062.jpg',
 'face_1066.jpg',
 'face_1076.jpg',
 'face_1075.jpg',
 'face_1074.jpg',
 'face_1069.jpg',
 'face_1078.jpg',
 'face_1079

In [0]:
images_train = list()
labels_train = list()
primary_path = data_dir + "corners/train/"
for i in range(len(col_file_train)):
  label = int(df_train['gender'][i])
  holder = col_file_train[i].split(".")
  name = holder[0] + "_corners.jpg"
  path = data_dir + "corners/train/" + name
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
  image_data = np.array(img)
  
  if image_data.shape == (0,0):
    continue 
  
  images_train.append(image_data)
  labels_train.append(label)

train_features = np.array(images_train)
labels_train = np.array(labels_train)
#y = np.array(keras.utils.to_categorical(np.array(labels_train), num_classes=2))
train_labels = np.array(keras.utils.to_categorical(np.array(labels_train), num_classes=2))
#train_features , test_features ,train_labels, test_labels = train_test_split( x_features , y , test_size=0.4 )

In [0]:
print(train_features.shape)
print(test_features.shape)
print(train_labels.shape)
print(test_labels.shape)

(1615, 128, 128)
(169, 128, 128)
(1615, 2)
(169, 2)


In [0]:
from tensorflow.python.keras import models , optimizers , losses ,activations
from tensorflow.python.keras.layers import *
import tensorflow as tf

In [0]:
class Classifier(object):
    def __init__( self , number_of_classes, dimensions):
        dropout_rate = 0.5
        self.dimensions = dimensions
        # input_shape = ( dimensions**2 , )
        input_shape = (128,128,1)
        convolution_shape = ( dimensions , dimensions , 1 )
        kernel_size = (3 , 3)
        pool_size = (2 , 2)
        strides = 1
        activation_func = activations.relu
        self.layers = [
            Reshape( input_shape=input_shape , target_shape=convolution_shape),
            Conv2D( 32, kernel_size=( 4 , 4 ) , strides=strides , activation=activation_func),
            MaxPooling2D(pool_size=pool_size, strides=strides ),
            Conv2D( 64, kernel_size=( 3 , 3 ) , strides=strides , activation=activation_func),
            MaxPooling2D(pool_size=pool_size , strides=strides),
            Flatten(input_shape=(128, 128, 1)),
            Dense( 100, activation=activation_func) ,
            Dropout(dropout_rate),
            Dense( number_of_classes, activation=tf.nn.softmax )
        ]
        self.model = tf.keras.Sequential(self.layers)
        # self.model.add(Flatten(input_shape=(128,128,1)))
        self.model.compile(
            optimizer=optimizers.Adam(lr=.001),
            loss=losses.categorical_crossentropy ,
            metrics=['accuracy'] ,
        )
    def fit(self, X, Y, hyperparameters):
        #if you want to not have that epoch printing you can just say verbose  = 0 in the .fit call 
        aug = ImageDataGenerator(
            rotation_range=20,
            zoom_range=0.50, ##
            width_shift_range=0.2, #0.2
            height_shift_range=0.2, #0.2
            shear_range=0.15,
            fill_mode="nearest",
            horizontal_flip=True)
        # self.model.fit(X, Y, batch_size=hyperparameters['batch_size'], epochs=hyperparameters['epochs'], shuffle = True)  
        eps = hyperparameters['epochs']
        bs = hyperparameters['batch_size']
        self.model.fit_generator(aug.flow(X, Y, batch_size=bs, shuffle = True), steps_per_epoch=math.ceil(len(X)/bs), epochs=eps)

    def evaluate(self , test_X , test_Y) :
        return self.model.evaluate(test_X, test_Y)

    def predict(self, X):
        predictions = self.model.predict(X)
        return predictions

    def save_model(self , file_path ):
        self.model.save(file_path )

    def load_model(self , file_path ):
        self.model = models.load_model(file_path)


In [0]:
data_dimension = 128
# train_features =  train_features.reshape( (  train_features.shape[0] , data_dimension**2  ) ).astype( np.float32 )
# test_features= test_features.reshape( ( test_features.shape[0] , data_dimension**2 ) ).astype( np.float32 )
r,c,d = train_features.shape
train_features = train_features.reshape((r,c,d,1))


print(train_features.shape)

print(train_labels.shape)


(1615, 128, 128, 1)
(1615, 2)


In [0]:

r,c,d = test_features.shape
test_features = test_features.reshape((r,c,d,1))


print(test_features.shape)

print(test_labels.shape)

(169, 128, 128, 1)
(169, 2)


Now to try and run 

In [0]:


classifier = Classifier( number_of_classes=2, dimensions = 128 )
parameters = {
    'batch_size' : 20 ,
    'epochs' : 8,
}

classifier.fit(train_features , train_labels  , hyperparameters=parameters )


# classifier.fit_generator(aug.flow(train_features , train_labels, batch_size=parameters.batch_size), steps_per_epoch=(len(train_features) / parameters.batch_size), epochs=parameters.epochs)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 1/8
81/81 [==============================] - 9s 111ms/step - loss: 91.0236 - acc: 0.5053
Epoch 2/8
81/81 [==============================] - 4s 49ms/step - loss: 0.6938 - acc: 0.5418
Epoch 3/8
81/81 [==============================] - 4s 48ms/step - loss: 0.6911 - acc: 0.5263
Epoch 4/8
81/81 [==============================] - 4s 49ms/step - loss: 0.6898 - acc: 0.5387
Epoch 5/8
81/81 [==============================] - 4s 48ms/step - loss: 0.6884 - acc: 0.5412
Epoch 6/8
81/81 [==============================] - 4s 49ms/step - loss: 0.6943 - acc: 0.5424
Epoch 7/8
81/81 [==============================] - 4s 48ms/step - loss: 0.6923 - acc: 0.5437
Epoch 8/8
81/81 [==============================] - 4s 48ms/step - loss: 0.6896 - acc: 0.5536


In [0]:
#classifier.save_model( 'models/model.h5')
preds = classifier.predict(test_features)
# loss , accuracy, FP, AUC, TP = classifier.evaluate( test_features , test_labels )
loss , accuracy = classifier.evaluate( test_features , test_labels )
# print( "Loss of {}".format( loss ) , "Accuracy of {} %".format( accuracy * 100 ), "False Positive {}".format( FP ), " TruePositive {}".format( TP ), "Area Under Curve{}".format( AUC ) )
print( "Loss of {}".format( loss ) , "Accuracy of {} %".format( accuracy * 100 ))

169/169 [==============================] - 0s 699us/sample - loss: 0.6903 - acc: 0.5385
Loss of 0.6903177972375993 Accuracy of 53.84615659713745 %




Training and Testing on Renaissance
* 25 epochs, train: 0.4554, test: 54.411762952804565 %
* zoom_range=0.50 epoch 25, train: 0.5644, test: 52.941179275512695 %

Training on BAM, testing on Renaissance
* 8 epochs, train:acc: 0.5536, test: 53.84615659713745 %